本教程将介绍如何使用管道（pipelines）让建模代码更整洁。

# 引言

**管道**是一种让数据预处理与建模代码保持条理的简单方式。具体而言，管道把预处理与建模步骤打包在一起，使你可以像调用单一步骤那样调用整个流程。

许多数据科学家在没有管道的情况下把模型“拼起来”，但管道具有重要优势：
1. **代码更整洁**：逐步处理数据常常让代码变得混乱；使用管道，你无需在每一步都手动维护训练/验证数据；
2. **更少的错误**：减少误用步骤或遗忘某个预处理步骤的机会；
3. **更易于生产化**：把原型模型迁移到可大规模部署的系统并不容易；虽然我们不展开这些话题，但管道能提供帮助；
4. **更多的验证方式**：下一节（交叉验证）会给出示例。

# 示例

与上一节一样，我们仍使用[墨尔本房价数据集](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)。

我们不展开数据加载细节，可假设已准备好 `X_train`、`X_valid`、`y_train` 和 `y_valid`。

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

我们用 `head()` 方法快速查看训练数据。注意其中既有类别数据，也有缺失值列；使用管道，这两类问题都能轻松处理！

In [ ]:
X_train.head()

我们分三步搭建完整管道。

### 第 1 步：定义预处理步骤

类似于管道，我们把预处理和建模捆绑在一起，用 `ColumnTransformer` 将不同的预处理步骤打包。下面的代码将：
- 对**数值型**数据进行缺失值插补；
- 对**类别型**数据进行缺失值插补并做独热编码。

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### 第 2 步：定义模型

接着，我们用熟悉的 [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) 定义一个随机森林模型。

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### 第 3 步：创建并评估管道

最后，我们使用 [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) 定义一个将预处理与建模打包在一起的管道。注意几点：
- 使用管道时，我们可一并完成训练数据的预处理与模型拟合。（否则就需要分别进行插补、独热编码与模型训练；这在同时包含数值与类别变量时将尤为繁琐。）
- 使用管道时，我们可以直接把未经预处理的 `X_valid` 传给 `predict()`，管道会在生成预测前自动完成预处理。（而没有管道时，要记得先手动预处理验证集再预测。）

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

# 结论

管道有助于清理机器学习代码、避免错误，尤其适合包含复杂数据预处理的工作流。

# 轮到你了

在[**下一道练习**](https://www.kaggle.com/kernels/fork/3370278)中使用管道，尝试更先进的数据预处理技术以提升预测效果！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*